# Use the logistic classifier to build a deep neural network


- TensorFlow code walkthrough to classify the letters in the MNIST database.
- The focus here is on the architecture of multilayer neural networks, not parameter tuning.

!['Deep neural network'](layers.png)

In [6]:
# Exploring data
import math
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.', one_hot=True, reshape=False)

b_features, _ = mnist.train.next_batch(10)
f = plt.figure(figsize=(20,12))
i = 1
for img in b_features:
    img2D =  img.reshape(28,28)
    f.add_subplot(2, 5, i)
    plt.imshow(img2D, cmap='gray')
    i = i + 1

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [7]:
# Parameters
n_epochs = 100
batch_size = 128  

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
n_hidden_layer = 256 # layer number of features

weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
x_flat = tf.reshape(x, [-1, n_input])

# Multilayer Perceptron

!['Multilayer Perceptron'](multi-layer.png)

In [12]:
keep_prob =tf.placeholder(tf.float32)
layer1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer1 = tf.nn.relu(layer1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)

logits = tf.add(tf.matmul(layer1, weights['out']), biases['out'])

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
opt  = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)

        for  i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(opt, feed_dict = {x: batch_features, 
                                       y: batch_labels,
                                       keep_prob: 0.5})

        if epoch % 10 == 0:
            valid_accuracy = sess.run(accuracy, feed_dict={x: mnist.validation.images, 
                                                           y: mnist.validation.labels,
                                                           keep_prob: 1.0})
            print('Epoch {:<3} - Accuracy: {}'.format(epoch, valid_accuracy))
            
    test_accuracy =  sess.run(accuracy, feed_dict={x: mnist.test.images, 
                                                   y: mnist.test.labels,
                                                   keep_prob: 1.0})
    print('Epoch {:<3} - Accuracy: {}'.format(epoch, test_accuracy))
    

Epoch 0   - Accuracy: 0.876800000667572
Epoch 10  - Accuracy: 0.902400016784668
Epoch 20  - Accuracy: 0.920799970626831
Epoch 30  - Accuracy: 0.9300000071525574
Epoch 40  - Accuracy: 0.9380000233650208
Epoch 50  - Accuracy: 0.9426000118255615
Epoch 60  - Accuracy: 0.944599986076355
Epoch 70  - Accuracy: 0.9506000280380249
Epoch 80  - Accuracy: 0.9539999961853027
Epoch 90  - Accuracy: 0.9527999758720398
Epoch 99  - Accuracy: 0.951200008392334


# Save and Restore TensorFlow Models

TensorFlow has the ability to save training progress using a class called `tf.train.Saver`. 
- This class provides the functionality to save any `tf.Variable` to file system.

### Saving Variables



In [ ]:
import tensorflow as tf

# The file path to save the data
save_file = './model2.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias    = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

    # Save the model
    saver.save(sess, save_file)

### Loading Variables
Now that the Tensor Variables are saved, let's load them back into a new model.

In [ ]:
# Remove the previous weights and bias
tf.reset_default_graph()

weights = tf.Variable(tf.truncated_normal([2, 3]))
bias    = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, save_file)

    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

# Save a Trained Model

Let's see how to train a model and save its weights.

In [ ]:
import math
tf.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

learning_rate = 0.1
n_input = 784  
n_classes = 10 

mnist = input_data.read_data_sets('.', one_hot=True)

features = tf.placeholder(tf.float32, [None, n_input])
labels   = tf.placeholder(tf.float32, [None, n_classes])

weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias    = tf.Variable(tf.random_normal([n_classes]))

logits = tf.add(tf.matmul(features, weights), bias)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
opt  = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# train model, then save the weights:
save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 200

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)

        for  i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(opt,feed_dict={features: batch_features, labels: batch_labels})

        if epoch % 10 == 0:
            valid_accuracy = sess.run(accuracy,
                                      feed_dict={
                                        features: mnist.validation.images,
                                        labels: mnist.validation.labels})
            
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                    epoch,valid_accuracy))
    saver.save(sess, save_file)
    print('Trained Model Saved.')